In [1]:
!pip install bindsnet

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.13.1+cu116 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1)
ERROR: No matching distribution found for torch==1.13.1+cu116


In [ ]:

# ⚠️ Установка BindsNET
#!pip install bindsnet

# 📦 Импорты
import torch
from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.network import Network
from bindsnet.network.nodes import Input, IzhikevichNodes
from bindsnet.network.topology import Connection
from bindsnet.learning import PostPre
from bindsnet.analysis.plotting import plot_spikes, plot_weights
from bindsnet.pipeline import EnvironmentPipeline
import matplotlib.pyplot as plt

# 🎯 Параметры
time = 100  # временное окно
n_hidden = 100
batch_size = 1

# 🧠 Инициализация сети
net = Network()

# Входной слой
input_layer = Input(n=784, shape=(1, 28, 28))
net.add_layer(input_layer, name='Input')

# Скрытый слой на нейронах Ижикевича
izh_layer = IzhikevichNodes(n=n_hidden)
net.add_layer(izh_layer, name='Izh')

# Связи и STDP
connection = Connection(source=input_layer, target=izh_layer)
net.add_connection(connection, source='Input', target='Izh')

# STDP-обучение
stdp = PostPre(connection=connection, nu=(0.01, -0.01))
net.add_learning_rule(stdp, connection=connection)

# 📚 Загрузка датасета MNIST
dataset = MNIST(encoding=PoissonEncoder(time=time), train=True, download=True)

# 🚀 Обучение и визуализация
for i in range(10):
    sample, label = dataset[i]
    sample = sample.view(time, 1, 784)
    inpts = {'Input': sample}
    net.run(inpts=inpts, time=time)

    # Визуализация после 1, 5, 10 итерации
    if i in [0, 4, 9]:
        # Спайки
        spikes = {name: layer.s for name, layer in net.layers.items()}
        plot_spikes(spikes)
        plt.suptitle(f"Spikes after sample {i+1}")
        plt.show()

        # Веса
        plot_weights(connection.w, wmax=1.0)
        plt.title(f"Weights after sample {i+1}")
        plt.show()

    net.reset_()
